In [2]:
%pylab inline
import pandas as pd
import seaborn

%config InlineBackend.figure_format = 'svg'
seaborn.set_style("white")

Populating the interactive namespace from numpy and matplotlib


In [12]:
df1 = pd.read_csv('data/turnstile_180623.txt')
df2 = pd.read_csv('data/turnstile_180602.txt')
df1.head(10)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/16/2018,00:00:00,REGULAR,6658725,2257032
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/16/2018,04:00:00,REGULAR,6658747,2257036
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/16/2018,08:00:00,REGULAR,6658766,2257068
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/16/2018,12:00:00,REGULAR,6658866,2257139
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/16/2018,16:00:00,REGULAR,6659043,2257188
5,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/16/2018,20:00:00,REGULAR,6659281,2257223
6,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/17/2018,00:00:00,REGULAR,6659424,2257246
7,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/17/2018,04:00:00,REGULAR,6659438,2257250
8,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/17/2018,08:00:00,REGULAR,6659449,2257263
9,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/17/2018,12:00:00,REGULAR,6659514,2257309


In [15]:
df_alldata = pd.concat([df1,df2])

The code below outputs a series of the differences in values between rows in the ENTRIES column. This is the number of people who entered the station in the last 4 hours.

In [16]:
df1['ENTRIES'].diff().head(10)

0      NaN
1     22.0
2     19.0
3    100.0
4    177.0
5    238.0
6    143.0
7     14.0
8     11.0
9     65.0
Name: ENTRIES, dtype: float64